In [233]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split, cross_val_score
import sklearn.metrics as metrics
import missingno as msno
import statsmodels.api as sm
from sklearn.preprocessing import PolynomialFeatures
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import Lasso, LassoCV

I will begin by importing the train and test data, adding a column that differentiates between both sets, concatenating the two data sets, cleaning the data, and adding transformations and dummy variables as needed.

I will then split the data into train and test before creating a model

# Data Import

In [234]:
# import train data
house_train = pd.read_csv('../datasets/train.csv')
house_train.head(2)

,Id,PID,MS SubClass,MS Zoning,Lot Frontage,Lot Area,Street,Alley,Lot Shape,Land Contour,...,Screen Porch,Pool Area,Pool QC,Fence,Misc Feature,Misc Val,Mo Sold,Yr Sold,Sale Type,SalePrice
0,109,533352170,60,RL,NaN,13517,Pave,NaN,IR1,Lvl,...,0,0,NaN,NaN,NaN,0,3,2010,WD,130500
1,544,531379050,60,RL,43.0,11492,Pave,NaN,IR1,Lvl,...,0,0,NaN,NaN,NaN,0,4,2009,WD,220000


In [235]:
# import test data
house_test = pd.read_csv('../datasets/test.csv')
house_test.head(2)

,Id,PID,MS SubClass,MS Zoning,Lot Frontage,Lot Area,Street,Alley,Lot Shape,Land Contour,...,3Ssn Porch,Screen Porch,Pool Area,Pool QC,Fence,Misc Feature,Misc Val,Mo Sold,Yr Sold,Sale Type
0,2658,902301120,190,RM,69.0,9142,Pave,Grvl,Reg,Lvl,...,0,0,0,NaN,NaN,NaN,0,4,2006,WD
1,2718,905108090,90,RL,NaN,9662,Pave,NaN,IR1,Lvl,...,0,0,0,NaN,NaN,NaN,0,8,2006,WD


In [236]:
# add column to differentiate between train (1) and test (0) data sets
house_train['data_type'] = 1
house_test['data_type'] = 0

In [237]:
# concatenate the data to clean/transform as needed
frames = [house_train, house_test]
df = pd.concat(frames)

In [238]:
# check that data was concatenated correctly
print(house_train.shape)
print(house_test.shape)
print(df.shape)
print(house_train.shape[0] + house_test.shape[0] == df.shape[0])

(2051, 82)
(878, 81)
(2929, 82)
True


# Cleaning Data and EDA

In [239]:
# When I plot the missing values for the full data set, train, and test separately, the graphs appear pretty similar. 
# Those null values may need to be zero instead.

In [240]:
features = ['Year Remod/Add', 'Yr Sold', 'Mo Sold', 'Neighborhood', 'MS Zoning', 'Overall Qual', 'Gr Liv Area', 'Garage Area', 'Garage Cars', 'Total Bsmt SF', 'Mas Vnr Area', '1st Flr SF', 'Year Built', 'Year Remod/Add', 'Full Bath', 'TotRms AbvGrd', 'Mo Sold', 'Yr Sold', 'SalePrice', 'data_type']
df = df[features]

In [241]:
df = pd.get_dummies(df, columns = ['MS Zoning', 'Neighborhood', 'Mo Sold', 'Yr Sold', 'Year Remod/Add'])

In [242]:
df_train = df[df['data_type'] == 1]
df_train.shape

(2051, 203)

In [247]:
df_train.isnull().sum()[df_train.isnull().sum()>0]

Garage Area       1
Garage Cars       1
Total Bsmt SF     1
Mas Vnr Area     22
dtype: int64

In [257]:
df_train = df_train.dropna()
df_train.shape

(2027, 203)

In [259]:
# values dropped
(2027-2051)/2051

-0.011701608971233545

In [260]:
df_test = df[df['data_type'] == 0]
df_test.shape

(878, 203)

In [265]:
y = df_train['SalePrice']
X = df_train.drop(columns = ['SalePrice', 'data_type'])

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 42)

y_train_log = y_train.map(np.log)
y_test_log = y_test.map(np.log)

lr = LinearRegression()
lr.fit(X_train,y_train_log)

y_pred_train = lr.predict(X_train)
y_pred_test = lr.predict(X_test)

mse_train = metrics.mean_squared_error(y_train_log, y_pred_train)
rmse_train = np.sqrt(mse_train)
print("rmse_train: ", rmse_train)

mse_test = metrics.mean_squared_error(y_test_log, y_pred_test)
rmse_test = np.sqrt(mse_test)
print("rmse_test: ", rmse_test)

cv_scores = cross_val_score(lr, X, y, cv=5).mean()
print('cv =', cv_scores)

rmse_train:  0.15813190220265963
rmse_test:  0.14358546473231923
cv = 0.8110437226298934


In [266]:
X_final = df_test.drop(columns = ['SalePrice', 'data_type'])

In [277]:
X_final = X_final.fillna(0)

Series([], dtype: int64)

In [278]:
SalePrice = lr.predict(X_final)

In [280]:
Answer = pd.DataFrame(house_test['Id'])
Answer['SalePrice'] = np.exp(SalePrice)
Answer.tail()

,Id,SalePrice
873,1662,176174.406206
874,1234,188793.301795
875,1373,132417.370218
876,1672,113843.167118
877,1939,117600.267734


In [281]:
Answer.to_csv('../datasets/submit_2.csv', index = False)

In [282]:
check = pd.read_csv('../datasets/submit_1.csv')

In [283]:
check.shape

(878, 2)

best predictors = continuous

MSSubClass: building class
MSZoning: general zoning classification
LotFrontage: Linear feet of street connected to property
LotArea: Lot size in square feet
Street: Type of road access to property
Alley: Type of alley access to property
LotShape: General shape of property
LandContour: Flatness of the property
Utilities: Type of utilities available
LotConfig: Lot configuration
LandSlope: Slope of property
Neighborhood: Physical locations within Ames city limits
Condition1: Proximity to main road or railroad
Condition2: Proximity to main road or railroad (if a second is present)
BldgType: Type of dwelling
HouseStyle: Style of dwelling
OverallQual: Overall material and finish quality
OverallCond: Overall condition rating
YearBuilt: Original construction date
YearRemodAdd: Remodel date (same as construction date if no remodeling or additions)
RoofStyle: Type of roof
RoofMatl: Roof material
Exterior1st: Exterior covering on house
Exterior2nd: Exterior covering on house (if more than one material)
MasVnrType: Masonry veneer type
MasVnrArea: Masonry veneer area in square feet
ExterQual: Exterior material quality
ExterCond: Present condition of the material on the exterior
Foundation: Type of foundation
BsmtQual: Height of the basement
BsmtCond: General condition of the basement
BsmtExposure: Walkout or garden level basement walls
BsmtFinType1: Quality of basement finished area
BsmtFinSF1: Type 1 finished square feet
BsmtFinType2: Quality of second finished area (if present)
BsmtFinSF2: Type 2 finished square feet
BsmtUnfSF: Unfinished square feet of basement area
TotalBsmtSF: Total square feet of basement area
Heating: Type of heating
HeatingQC: Heating quality and condition
CentralAir: Central air conditioning
Electrical: Electrical system
1stFlrSF: First Floor square feet
2ndFlrSF: Second floor square feet
LowQualFinSF: Low quality finished square feet (all floors)
GrLivArea: Above grade (ground) living area square feet
BsmtFullBath: Basement full bathrooms
BsmtHalfBath: Basement half bathrooms
FullBath: Full bathrooms above grade
HalfBath: Half baths above grade
Bedroom: Number of bedrooms above basement level
Kitchen: Number of kitchens
KitchenQual: Kitchen quality
TotRmsAbvGrd: Total rooms above grade (does not include bathrooms)
Functional: Home functionality rating
Fireplaces: Number of fireplaces
FireplaceQu: Fireplace quality
GarageType: Garage location
GarageYrBlt: Year garage was built
GarageFinish: Interior finish of the garage
GarageCars: Size of garage in car capacity
GarageArea: Size of garage in square feet
GarageQual: Garage quality
GarageCond: Garage condition
PavedDrive: Paved driveway
WoodDeckSF: Wood deck area in square feet
OpenPorchSF: Open porch area in square feet
EnclosedPorch: Enclosed porch area in square feet
3SsnPorch: Three season porch area in square feet
ScreenPorch: Screen porch area in square feet
PoolArea: Pool area in square feet
PoolQC: Pool quality
Fence: Fence quality
MiscFeature: Miscellaneous feature not covered in other categories
MiscVal: $Value of miscellaneous feature
MoSold: Month Sold
YrSold: Year Sold
SaleType: Type of sale